# Restructuring Data into a Tidy Form

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 4, 'max_rows', 10, 'max_colwidth', 12)

## Introduction

## Tidying variable values as column names with stack

In [2]:
state_fruit = pd.read_csv('data/state_fruit.csv', index_col=0)
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


### How to do it...

In [3]:
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [4]:
(state_fruit
   .stack()
   .reset_index()
)

,level_0,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [5]:
(state_fruit
   .stack()
   .reset_index()
   .rename(columns={'level_0':'state', 
      'level_1': 'fruit', 0: 'weight'})
)

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [6]:
(state_fruit
    .stack()
    .rename_axis(['state', 'fruit'])
)

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

In [7]:
(state_fruit
    .stack()
    .rename_axis(['state', 'fruit'])
    .reset_index(name='weight')
)

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


### How it works...

### There's more...

In [8]:
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [9]:
state_fruit2.stack()

0  State       Texas
   Apple          12
   Orange         10
   Banana         40
1  State     Arizona
              ...   
   Banana         12
2  State     Florida
   Apple           0
   Orange         14
   Banana        190
Length: 12, dtype: object

In [10]:
state_fruit2.set_index('State').stack()

State          
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

## Tidying variable values as column names with melt

### How to do it...

In [11]:
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [12]:
state_fruit2.melt(id_vars=['State'],
    value_vars=['Apple', 'Orange', 'Banana'])

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [13]:
state_fruit2.melt(id_vars=['State'],
                   value_vars=['Apple', 'Orange', 'Banana'],
                   var_name='Fruit',
                   value_name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


### How it works...

### There's more...

In [14]:
state_fruit2.melt()

,variable,value
0,State,Texas
1,State,Arizona
2,State,Florida
3,Apple,12
4,Apple,9
...,...,...
7,Orange,7
8,Orange,14
9,Banana,40
10,Banana,12


In [15]:
state_fruit2.melt(id_vars='State')

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


## Stacking multiple groups of variables simultaneously

In [16]:
movie = pd.read_csv('data/movie.csv')
actor = movie[['movie_title', 'actor_1_name',
               'actor_2_name', 'actor_3_name',
               'actor_1_facebook_likes',
               'actor_2_facebook_likes',
               'actor_3_facebook_likes']]
actor.head()

,movie_title,actor_1_name,...,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,...,936.0,855.0
1,Pirates ...,Johnny Depp,...,5000.0,1000.0
2,Spectre,Christop...,...,393.0,161.0
3,The Dark...,Tom Hardy,...,23000.0,23000.0
4,Star War...,Doug Walker,...,12.0,NaN


### How to do it...

In [17]:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = (col_name[:5] + col_name[fb_idx - 1:] 
               + col_name[5:fb_idx-1])
    return col_name

In [18]:
actor2 = actor.rename(columns=change_col_name)
actor2

,movie_title,actor_1,...,actor_facebook_likes_2,actor_facebook_likes_3
0,Avatar,CCH Pounder,...,936.0,855.0
1,Pirates ...,Johnny Depp,...,5000.0,1000.0
2,Spectre,Christop...,...,393.0,161.0
3,The Dark...,Tom Hardy,...,23000.0,23000.0
4,Star War...,Doug Walker,...,12.0,NaN
...,...,...,...,...,...
4911,Signed S...,Eric Mabius,...,470.0,318.0
4912,The Foll...,Natalie Zea,...,593.0,319.0
4913,A Plague...,Eva Boehnke,...,0.0,0.0
4914,Shanghai...,Alan Ruck,...,719.0,489.0


In [19]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2,
    stubnames=stubs,
    i=['movie_title'],
    j='actor_num',
    sep='_')
actor2_tidy.head()

,,actor,actor_facebook_likes
movie_title,actor_num,,
Avatar,1,CCH Pounder,1000.0
Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
Spectre,1,Christop...,11000.0
The Dark Knight Rises,1,Tom Hardy,27000.0
Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0


### How it works...

### There's more...

In [20]:
df = pd.read_csv('data/stackme.csv')
df

,State,Country,...,d,e
0,TX,US,...,2,6
1,MA,US,...,9,7
2,ON,CAN,...,4,2


In [21]:
df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                     'd':'group2_a1', 'e':'group2_b2'})

,State,Country,...,group2_a1,group2_b2
0,TX,US,...,2,6
1,MA,US,...,9,7
2,ON,CAN,...,4,2


In [22]:
pd.wide_to_long(
       df.rename(columns = {'a1':'group1_a1', 
                 'b2':'group1_b2',
                 'd':'group2_a1', 'e':'group2_b2'}),
    stubnames=['group1', 'group2'],
    i=['State', 'Country', 'Test'],
    j='Label',
    suffix='.+',
    sep='_')

group1  group2
State Country Test  Label                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

## Inverting stacked data

### How to do it...

In [23]:
usecol_func = lambda x: 'UGDS_' in x or x == 'INSTNM'
college = pd.read_csv('data/college.csv',
    index_col='INSTNM',
    usecols=usecol_func)
college

,UGDS_WHITE,UGDS_BLACK,...,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,
Alabama A & M University,0.0333,0.9353,...,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,...,0.0179,0.0100
Amridge University,0.2990,0.4192,...,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,...,0.0332,0.0350
Alabama State University,0.0158,0.9208,...,0.0243,0.0137
...,...,...,...,...,...
SAE Institute of Technology San Francisco,NaN,NaN,...,NaN,NaN
Rasmussen College - Overland Park,NaN,NaN,...,NaN,NaN
National Personal Training Institute of Cleveland,NaN,NaN,...,NaN,NaN


In [24]:
college_stacked = college.stack()
college_stacked

INSTNM                                     
Alabama A & M University         UGDS_WHITE    0.0333
                                 UGDS_BLACK    0.9353
                                 UGDS_HISP     0.0055
                                 UGDS_ASIAN    0.0019
                                 UGDS_AIAN     0.0024
                                                ...  
Coastal Pines Technical College  UGDS_AIAN     0.0034
                                 UGDS_NHPI     0.0017
                                 UGDS_2MOR     0.0191
                                 UGDS_NRA      0.0028
                                 UGDS_UNKN     0.0056
Length: 61866, dtype: float64

In [25]:
college_stacked.unstack()

,UGDS_WHITE,UGDS_BLACK,...,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,
Alabama A & M University,0.0333,0.9353,...,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,...,0.0179,0.0100
Amridge University,0.2990,0.4192,...,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,...,0.0332,0.0350
Alabama State University,0.0158,0.9208,...,0.0243,0.0137
...,...,...,...,...,...
Hollywood Institute of Beauty Careers-West Palm Beach,0.2182,0.4182,...,0.0182,0.0909
Hollywood Institute of Beauty Careers-Casselberry,0.1200,0.3333,...,0.0000,0.0667
Coachella Valley Beauty College-Beaumont,0.3284,0.1045,...,0.0000,0.0000


In [26]:
college2 = pd.read_csv('data/college.csv',
   usecols=usecol_func)
college2

,INSTNM,UGDS_WHITE,...,UGDS_NRA,UGDS_UNKN
0,Alabama ...,0.0333,...,0.0059,0.0138
1,Universi...,0.5922,...,0.0179,0.0100
2,Amridge ...,0.2990,...,0.0000,0.2715
3,Universi...,0.6988,...,0.0332,0.0350
4,Alabama ...,0.0158,...,0.0243,0.0137
...,...,...,...,...,...
7530,SAE Inst...,NaN,...,NaN,NaN
7531,Rasmusse...,NaN,...,NaN,NaN
7532,National...,NaN,...,NaN,NaN
7533,Bay Area...,NaN,...,NaN,NaN


In [27]:
college_melted = college2.melt(id_vars='INSTNM',
    var_name='Race',
    value_name='Percentage')
college_melted

,INSTNM,Race,Percentage
0,Alabama ...,UGDS_WHITE,0.0333
1,Universi...,UGDS_WHITE,0.5922
2,Amridge ...,UGDS_WHITE,0.2990
3,Universi...,UGDS_WHITE,0.6988
4,Alabama ...,UGDS_WHITE,0.0158
...,...,...,...
67810,SAE Inst...,UGDS_UNKN,NaN
67811,Rasmusse...,UGDS_UNKN,NaN
67812,National...,UGDS_UNKN,NaN
67813,Bay Area...,UGDS_UNKN,NaN


In [28]:
melted_inv = college_melted.pivot(index='INSTNM',
    columns='Race',
    values='Percentage')
melted_inv

Race,UGDS_2MOR,UGDS_AIAN,...,UGDS_UNKN,UGDS_WHITE
INSTNM,,,,,
A & W Healthcare Educators,0.0000,0.0000,...,0.0000,0.0000
A T Still University of Health Sciences,NaN,NaN,...,NaN,NaN
ABC Beauty Academy,0.0000,0.0000,...,0.0000,0.0000
ABC Beauty College Inc,0.0000,0.0000,...,0.0000,0.2895
AI Miami International University of Art and Design,0.0018,0.0000,...,0.4644,0.0324
...,...,...,...,...,...
Yukon Beauty College Inc,0.0000,0.1200,...,0.0000,0.8000
Z Hair Academy,0.0211,0.0000,...,0.0105,0.9368
Zane State College,0.0218,0.0029,...,0.2399,0.6995


In [29]:
college2_replication = (melted_inv
    .loc[college2['INSTNM'], college2.columns[1:]]
    .reset_index()
)
college2.equals(college2_replication)

True

### How it works...

### There's more...

In [30]:
college.stack().unstack(0)

INSTNM,Alabama A & M University,University of Alabama at Birmingham,...,Dewey University-Mayaguez,Coastal Pines Technical College
UGDS_WHITE,0.0333,0.5922,...,0.0,0.6762
UGDS_BLACK,0.9353,0.2600,...,0.0,0.2508
UGDS_HISP,0.0055,0.0283,...,1.0,0.0359
UGDS_ASIAN,0.0019,0.0518,...,0.0,0.0045
UGDS_AIAN,0.0024,0.0022,...,0.0,0.0034
UGDS_NHPI,0.0019,0.0007,...,0.0,0.0017
UGDS_2MOR,0.0000,0.0368,...,0.0,0.0191
UGDS_NRA,0.0059,0.0179,...,0.0,0.0028
UGDS_UNKN,0.0138,0.0100,...,0.0,0.0056


In [31]:
college.T
college.transpose()

INSTNM,Alabama A & M University,University of Alabama at Birmingham,...,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,...,NaN,NaN
UGDS_BLACK,0.9353,0.2600,...,NaN,NaN
UGDS_HISP,0.0055,0.0283,...,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,...,NaN,NaN
UGDS_AIAN,0.0024,0.0022,...,NaN,NaN
UGDS_NHPI,0.0019,0.0007,...,NaN,NaN
UGDS_2MOR,0.0000,0.0368,...,NaN,NaN
UGDS_NRA,0.0059,0.0179,...,NaN,NaN
UGDS_UNKN,0.0138,0.0100,...,NaN,NaN


## Unstacking after a groupby aggregation

### How to do it...

In [32]:
employee = pd.read_csv('data/employee.csv')
(employee
    .groupby('RACE')
    ['BASE_SALARY']
    .mean()
    .astype(int)
)

RACE
American Indian or Alaskan Native    60272
Asian/Pacific Islander               61660
Black or African American            50137
Hispanic/Latino                      52345
Others                               51278
White                                64419
Name: BASE_SALARY, dtype: int64

In [33]:
(employee
    .groupby(['RACE', 'GENDER'])
    ['BASE_SALARY'] 
    .mean()
    .astype(int)
)

RACE                               GENDER
American Indian or Alaskan Native  Female    60238
                                   Male      60305
Asian/Pacific Islander             Female    63226
                                   Male      61033
Black or African American          Female    48915
                                             ...  
Hispanic/Latino                    Male      54782
Others                             Female    63785
                                   Male      38771
White                              Female    66793
                                   Male      63940
Name: BASE_SALARY, Length: 12, dtype: int64

In [34]:
(employee
    .groupby(['RACE', 'GENDER'])
    ['BASE_SALARY'] 
    .mean()
    .astype(int)
    .unstack('GENDER')
)

GENDER,Female,Male
RACE,,
American Indian or Alaskan Native,60238,60305
Asian/Pacific Islander,63226,61033
Black or African American,48915,51082
Hispanic/Latino,46503,54782
Others,63785,38771
White,66793,63940


In [35]:
(employee
    .groupby(['RACE', 'GENDER'])
    ['BASE_SALARY'] 
    .mean()
    .astype(int)
    .unstack('RACE')
)

RACE,American Indian or Alaskan Native,Asian/Pacific Islander,...,Others,White
GENDER,,,,,
Female,60238,63226,...,63785,66793
Male,60305,61033,...,38771,63940


### How it works...

### There's more...

In [36]:
(employee
    .groupby(['RACE', 'GENDER'])
    ['BASE_SALARY']
    .agg(['mean', 'max', 'min'])
    .astype(int)
)

mean     max    min
RACE         GENDER                      
American ... Female  60238   98536  26125
             Male    60305   81239  26125
Asian/Pac... Female  63226  130416  26125
             Male    61033  163228  27914
Black or ... Female  48915  150416  24960
...                    ...     ...    ...
Hispanic/... Male    54782  165216  26104
Others       Female  63785   63785  63785
             Male    38771   38771  38771
White        Female  66793  178331  27955
             Male    63940  210588  26125

[12 rows x 3 columns]

In [37]:
(employee
    .groupby(['RACE', 'GENDER'])
    ['BASE_SALARY']
    .agg(['mean', 'max', 'min'])
    .astype(int)
    .unstack('GENDER')
)

mean         ...    min       
GENDER       Female   Male  ... Female   Male
RACE                        ...              
American ...  60238  60305  ...  26125  26125
Asian/Pac...  63226  61033  ...  26125  27914
Black or ...  48915  51082  ...  24960  26125
Hispanic/...  46503  54782  ...  26125  26104
Others        63785  38771  ...  63785  38771
White         66793  63940  ...  27955  26125

[6 rows x 6 columns]

## Replicating pivot_table with a groupby aggregation

### How to do it...

In [38]:
flights = pd.read_csv('data/flights.csv')
fpt = flights.pivot_table(index='AIRLINE',
    columns='ORG_AIR',
    values='CANCELLED',
    aggfunc='sum',
    fill_value=0).round(2)
fpt

ORG_AIR,ATL,DEN,...,PHX,SFO
AIRLINE,,,,,
AA,3,4,...,4,2
AS,0,0,...,0,0
B6,0,0,...,0,1
DL,28,1,...,1,2
EV,18,6,...,0,0
...,...,...,...,...,...
OO,3,25,...,9,33
UA,2,9,...,3,19
US,0,0,...,7,3


In [39]:
(flights
    .groupby(['AIRLINE', 'ORG_AIR'])
    ['CANCELLED']
    .sum()
)

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
                    ..
WN       LAS         7
         LAX        32
         MSP         1
         PHX         6
         SFO        25
Name: CANCELLED, Length: 114, dtype: int64

In [40]:
fpg = (flights
    .groupby(['AIRLINE', 'ORG_AIR'])
    ['CANCELLED']
    .sum()
    .unstack('ORG_AIR', fill_value=0)
)

In [41]:
fpt.equals(fpg)

True

### How it works...

### There's more...

In [42]:
flights.pivot_table(index=['AIRLINE', 'MONTH'],
    columns=['ORG_AIR', 'CANCELLED'],
    values=['DEP_DELAY', 'DIST'],
    aggfunc=['sum', 'mean'],
    fill_value=0)

sum     ...         mean        
              DEP_DELAY     ...         DIST        
ORG_AIR             ATL     ...          SFO        
CANCELLED             0  1  ...            0       1
AIRLINE MONTH               ...                     
AA      1           -13  0  ...  1860.166667     0.0
        2           -39  0  ...  1337.916667  2586.0
        3            -2  0  ...  1502.758621     0.0
        4             1  0  ...  1646.903226     0.0
        5            52  0  ...  1436.892857     0.0
...                 ... ..  ...          ...     ...
WN      7          2604  0  ...   636.210526     0.0
        8          1718  0  ...   644.857143   392.0
        9          1033  0  ...   731.578947   354.5
        11          700  0  ...   580.875000   392.0
        12         1679  0  ...   782.256410     0.0

[149 rows x 80 columns]

In [43]:
(flights
    .groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED']) 
    ['DEP_DELAY', 'DIST'] 
    .agg(['mean', 'sum']) 
    .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0) 
    .swaplevel(0, 1, axis='columns')
)

mean      ...      sum        
               DEP_DELAY      ...     DIST        
ORG_AIR              ATL      ...      SFO        
CANCELLED              0   1  ...        0       1
AIRLINE MONTH                 ...                 
AA      1      -3.250000 NaN  ...  33483.0     NaN
        2      -3.000000 NaN  ...  32110.0  2586.0
        3      -0.166667 NaN  ...  43580.0     NaN
        4       0.071429 NaN  ...  51054.0     NaN
        5       5.777778 NaN  ...  40233.0     NaN
...                  ...  ..  ...      ...     ...
WN      7      21.700000 NaN  ...  24176.0     NaN
        8      16.207547 NaN  ...  18056.0   784.0
        9       8.680672 NaN  ...  27800.0   709.0
        11      5.932203 NaN  ...  23235.0   784.0
        12     15.691589 NaN  ...  30508.0     NaN

[149 rows x 80 columns]

## Renaming axis levels for easy reshaping

### How to do it...

In [44]:
college = pd.read_csv('data/college.csv')
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
)

UGDS         ... SATMTMID       
                size    min  ...      min    max
STABBR RELAFFIL              ...                
AK     0           7  109.0  ...      NaN    NaN
       1           3   27.0  ...    503.0  503.0
AL     0          72   12.0  ...    420.0  590.0
       1          24   13.0  ...    400.0  560.0
AR     0          68   18.0  ...    427.0  565.0
...              ...    ...  ...      ...    ...
WI     0          87   20.0  ...    480.0  680.0
       1          25    4.0  ...    452.0  605.0
WV     0          65   20.0  ...    430.0  530.0
       1           8   63.0  ...    455.0  510.0
WY     0          11   52.0  ...    540.0  540.0

[112 rows x 6 columns]

In [45]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
)

AGG_COLS        UGDS         ... SATMTMID       
AGG_FUNCS       size    min  ...      min    max
STABBR RELAFFIL              ...                
AK     0           7  109.0  ...      NaN    NaN
       1           3   27.0  ...    503.0  503.0
AL     0          72   12.0  ...    420.0  590.0
       1          24   13.0  ...    400.0  560.0
AR     0          68   18.0  ...    427.0  565.0
...              ...    ...  ...      ...    ...
WI     0          87   20.0  ...    480.0  680.0
       1          25    4.0  ...    452.0  605.0
WV     0          65   20.0  ...    430.0  530.0
       1           8   63.0  ...    455.0  510.0
WY     0          11   52.0  ...    540.0  540.0

[112 rows x 6 columns]

In [46]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
    .stack('AGG_FUNCS')
)

AGG_COLS                      UGDS  SATMTMID
STABBR RELAFFIL AGG_FUNCS                   
AK     0        size           7.0       7.0
                min          109.0       NaN
                max        12865.0       NaN
       1        size           3.0       3.0
                min           27.0     503.0
...                            ...       ...
WV     1        min           63.0     455.0
                max         1375.0     510.0
WY     0        size          11.0      11.0
                min           52.0     540.0
                max         9910.0     540.0

[332 rows x 2 columns]

In [47]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
    .stack('AGG_FUNCS')
    .swaplevel('AGG_FUNCS', 'STABBR',
       axis='index')
)

AGG_COLS                      UGDS  SATMTMID
AGG_FUNCS RELAFFIL STABBR                   
size      0        AK          7.0       7.0
min       0        AK        109.0       NaN
max       0        AK      12865.0       NaN
size      1        AK          3.0       3.0
min       1        AK         27.0     503.0
...                            ...       ...
                   WV         63.0     455.0
max       1        WV       1375.0     510.0
size      0        WY         11.0      11.0
min       0        WY         52.0     540.0
max       0        WY       9910.0     540.0

[332 rows x 2 columns]

In [48]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
    .stack('AGG_FUNCS')
    .swaplevel('AGG_FUNCS', 'STABBR', axis='index') 
    .sort_index(level='RELAFFIL', axis='index') 
    .sort_index(level='AGG_COLS', axis='columns')
)

AGG_COLS                   SATMTMID      UGDS
AGG_FUNCS RELAFFIL STABBR                    
max       0        AK           NaN   12865.0
                   AL         590.0   29851.0
                   AR         565.0   21405.0
                   AS           NaN    1276.0
                   AZ         580.0  151558.0
...                             ...       ...
size      1        VI           1.0       1.0
                   VT           5.0       5.0
                   WA          17.0      17.0
                   WI          25.0      25.0
                   WV           8.0       8.0

[332 rows x 2 columns]

In [49]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
    .stack('AGG_FUNCS')
    .unstack(['RELAFFIL', 'STABBR'])
)

AGG_COLS      UGDS         ... SATMTMID       
RELAFFIL         0      1  ...        1      0
STABBR          AK     AK  ...       WV     WY
AGG_FUNCS                  ...                
size           7.0    3.0  ...      8.0   11.0
min          109.0   27.0  ...    455.0  540.0
max        12865.0  275.0  ...    510.0  540.0

[3 rows x 224 columns]

In [50]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
    .stack(['AGG_FUNCS', 'AGG_COLS'])
)

STABBR  RELAFFIL  AGG_FUNCS  AGG_COLS
AK      0         size       UGDS            7.0
                             SATMTMID        7.0
                  min        UGDS          109.0
                  max        UGDS        12865.0
        1         size       UGDS            3.0
                                          ...   
WY      0         size       SATMTMID       11.0
                  min        UGDS           52.0
                             SATMTMID      540.0
                  max        UGDS         9910.0
                             SATMTMID      540.0
Length: 640, dtype: float64

In [51]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
    .unstack(['STABBR', 'RELAFFIL']) 
)

AGG_COLS  AGG_FUNCS  STABBR  RELAFFIL
UGDS      size       AK      0             7.0
                             1             3.0
                     AL      0            72.0
                             1            24.0
                     AR      0            68.0
                                         ...  
SATMTMID  max        WI      1           605.0
                     WV      0           530.0
                             1           510.0
                     WY      0           540.0
                             1             NaN
Length: 708, dtype: float64

### How it works...

### There's more...

In [52]:
(college
    .groupby(['STABBR', 'RELAFFIL']) 
    ['UGDS', 'SATMTMID'] 
    .agg(['size', 'min', 'max'])
    .rename_axis([None, None], axis='index') 
    .rename_axis([None, None], axis='columns')
)

UGDS         ... SATMTMID       
     size    min  ...      min    max
AK 0    7  109.0  ...      NaN    NaN
   1    3   27.0  ...    503.0  503.0
AL 0   72   12.0  ...    420.0  590.0
   1   24   13.0  ...    400.0  560.0
AR 0   68   18.0  ...    427.0  565.0
...   ...    ...  ...      ...    ...
WI 0   87   20.0  ...    480.0  680.0
   1   25    4.0  ...    452.0  605.0
WV 0   65   20.0  ...    430.0  530.0
   1    8   63.0  ...    455.0  510.0
WY 0   11   52.0  ...    540.0  540.0

[112 rows x 6 columns]

## Tidying when multiple variables are stored as column names

### How to do it...

In [53]:
weightlifting = pd.read_csv('data/weightlifting_men.csv')
weightlifting

,Weight Category,M35 35-39,...,M75 75-79,M80 80+
0,56,137,...,62,55
1,62,152,...,67,57
2,69,167,...,75,60
3,77,182,...,82,65
4,85,192,...,87,70
5,94,202,...,90,75
6,105,210,...,95,80
7,105+,217,...,100,85


In [54]:
(weightlifting
    .melt(id_vars='Weight Category',
          var_name='sex_age',
          value_name='Qual Total')
)

,Weight Category,sex_age,Qual Total
0,56,M35 35-39,137
1,62,M35 35-39,152
2,69,M35 35-39,167
3,77,M35 35-39,182
4,85,M35 35-39,192
...,...,...,...
75,77,M80 80+,65
76,85,M80 80+,70
77,94,M80 80+,75
78,105,M80 80+,80


In [55]:
(weightlifting
    .melt(id_vars='Weight Category',
          var_name='sex_age',
          value_name='Qual Total')
    ['sex_age']
    .str.split(expand=True)
)

,0,1
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39
...,...,...
75,M80,80+
76,M80,80+
77,M80,80+
78,M80,80+


In [56]:
(weightlifting
    .melt(id_vars='Weight Category',
          var_name='sex_age',
          value_name='Qual Total')
    ['sex_age']
    .str.split(expand=True)
    .rename(columns={0:'Sex', 1:'Age Group'})
)

,Sex,Age Group
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39
...,...,...
75,M80,80+
76,M80,80+
77,M80,80+
78,M80,80+


In [57]:
(weightlifting
    .melt(id_vars='Weight Category',
          var_name='sex_age',
          value_name='Qual Total')
    ['sex_age']
    .str.split(expand=True)
    .rename(columns={0:'Sex', 1:'Age Group'})
    .assign(Sex=lambda df_: df_.Sex.str[0])
)

,Sex,Age Group
0,M,35-39
1,M,35-39
2,M,35-39
3,M,35-39
4,M,35-39
...,...,...
75,M,80+
76,M,80+
77,M,80+
78,M,80+


In [58]:
melted = (weightlifting
    .melt(id_vars='Weight Category',
          var_name='sex_age',
          value_name='Qual Total')
)
tidy = pd.concat([melted
           ['sex_age']
           .str.split(expand=True)
           .rename(columns={0:'Sex', 1:'Age Group'})
           .assign(Sex=lambda df_: df_.Sex.str[0]),
          melted[['Weight Category', 'Qual Total']]],
          axis='columns'
)
tidy

,Sex,Age Group,Weight Category,Qual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192
...,...,...,...,...
75,M,80+,77,65
76,M,80+,85,70
77,M,80+,94,75
78,M,80+,105,80


In [59]:
melted = (weightlifting
    .melt(id_vars='Weight Category',
          var_name='sex_age',
          value_name='Qual Total')
)
(melted
    ['sex_age']
    .str.split(expand=True)
    .rename(columns={0:'Sex', 1:'Age Group'})
    .assign(Sex=lambda df_: df_.Sex.str[0],
            Category=melted['Weight Category'],
            Total=melted['Qual Total'])
)

,Sex,Age Group,Category,Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192
...,...,...,...,...
75,M,80+,77,65
76,M,80+,85,70
77,M,80+,94,75
78,M,80+,105,80


### How it works...

### There's more...

In [60]:
tidy2 = (weightlifting
    .melt(id_vars='Weight Category',
          var_name='sex_age',
          value_name='Qual Total')
    .assign(Sex=lambda df_:df_.sex_age.str[0],
            **{'Age Group':(lambda df_: (df_
                .sex_age
                .str.extract(r'(\d{2}[-+](?:\d{2})?)',
                             expand=False)))})
    .drop(columns='sex_age')
)

In [61]:
tidy2

,Weight Category,Qual Total,Sex,Age Group
0,56,137,M,35-39
1,62,152,M,35-39
2,69,167,M,35-39
3,77,182,M,35-39
4,85,192,M,35-39
...,...,...,...,...
75,77,65,M,80+
76,85,70,M,80+
77,94,75,M,80+
78,105,80,M,80+


In [62]:
tidy.sort_index(axis=1).equals(tidy2.sort_index(axis=1))

True

## Tidying when multiple variables are stored is a single column

### How to do it...

In [63]:
inspections = pd.read_csv('data/restaurant_inspections.csv',
    parse_dates=['Date'])
inspections

,Name,Date,Info,Value
0,E & E Gr...,2017-08-08,Borough,MANHATTAN
1,E & E Gr...,2017-08-08,Cuisine,American
2,E & E Gr...,2017-08-08,Description,Non-food...
3,E & E Gr...,2017-08-08,Grade,A
4,E & E Gr...,2017-08-08,Score,9.0
...,...,...,...,...
495,PIER SIX...,2017-09-01,Borough,MANHATTAN
496,PIER SIX...,2017-09-01,Cuisine,American
497,PIER SIX...,2017-09-01,Description,Filth fl...
498,PIER SIX...,2017-09-01,Grade,Z


In [64]:
inspections.pivot(index=['Name', 'Date'],
    columns='Info', values='Value')

NotImplementedError: > 1 ndim Categorical are not supported at this time

In [65]:
inspections.set_index(['Name','Date', 'Info'])

Value
Name         Date       Info                    
E & E Gri... 2017-08-08 Borough        MANHATTAN
                        Cuisine         American
                        Description  Non-food...
                        Grade                  A
                        Score                9.0
...                                          ...
PIER SIXT... 2017-09-01 Borough        MANHATTAN
                        Cuisine         American
                        Description  Filth fl...
                        Grade                  Z
                        Score               33.0

[500 rows x 1 columns]

In [66]:
(inspections
    .set_index(['Name','Date', 'Info']) 
    .unstack('Info')
)

Value               ...            
Info                         Borough      Cuisine  ... Grade Score
Name         Date                                  ...            
3 STAR JU... 2017-05-10     BROOKLYN  Juice, S...  ...     A  12.0
A & L PIZ... 2017-08-22     BROOKLYN        Pizza  ...     A   9.0
AKSARAY T... 2017-07-25     BROOKLYN      Turkish  ...     A  13.0
ANTOJITOS... 2017-06-01     BROOKLYN  Latin (C...  ...     A  10.0
BANGIA       2017-06-16    MANHATTAN       Korean  ...     A   9.0
...                              ...          ...  ...   ...   ...
VALL'S PI... 2017-03-15  STATEN I...  Pizza/It...  ...     A   9.0
VIP GRILL    2017-06-12     BROOKLYN  Jewish/K...  ...     A  10.0
WAHIZZA      2017-04-13    MANHATTAN        Pizza  ...     A  10.0
WANG MAND... 2017-08-29       QUEENS       Korean  ...     A  12.0
XIAOYAN Y... 2017-08-29       QUEENS       Korean  ...     Z  49.0

[100 rows x 5 columns]

In [67]:
(inspections
    .set_index(['Name','Date', 'Info']) 
    .unstack('Info')
    .reset_index(col_level=-1)
)

... Value      
Info         Name       Date  ... Grade Score
0     3 STAR J... 2017-05-10  ...     A  12.0
1     A & L PI... 2017-08-22  ...     A   9.0
2     AKSARAY ... 2017-07-25  ...     A  13.0
3     ANTOJITO... 2017-06-01  ...     A  10.0
4          BANGIA 2017-06-16  ...     A   9.0
..            ...        ...  ...   ...   ...
95    VALL'S P... 2017-03-15  ...     A   9.0
96      VIP GRILL 2017-06-12  ...     A  10.0
97        WAHIZZA 2017-04-13  ...     A  10.0
98    WANG MAN... 2017-08-29  ...     A  12.0
99    XIAOYAN ... 2017-08-29  ...     Z  49.0

[100 rows x 7 columns]

In [68]:
def flatten0(df_):
    df_.columns = df_.columns.droplevel(0).rename(None)
    return df_

In [69]:
(inspections
    .set_index(['Name','Date', 'Info']) 
    .unstack('Info')
    .reset_index(col_level=-1)
    .pipe(flatten0)
)

,Name,Date,...,Grade,Score
0,3 STAR J...,2017-05-10,...,A,12.0
1,A & L PI...,2017-08-22,...,A,9.0
2,AKSARAY ...,2017-07-25,...,A,13.0
3,ANTOJITO...,2017-06-01,...,A,10.0
4,BANGIA,2017-06-16,...,A,9.0
...,...,...,...,...,...
95,VALL'S P...,2017-03-15,...,A,9.0
96,VIP GRILL,2017-06-12,...,A,10.0
97,WAHIZZA,2017-04-13,...,A,10.0
98,WANG MAN...,2017-08-29,...,A,12.0


In [70]:
(inspections
    .set_index(['Name','Date', 'Info']) 
    .squeeze() 
    .unstack('Info') 
    .reset_index() 
    .rename_axis(None, axis='columns')
)

,Name,Date,...,Grade,Score
0,3 STAR J...,2017-05-10,...,A,12.0
1,A & L PI...,2017-08-22,...,A,9.0
2,AKSARAY ...,2017-07-25,...,A,13.0
3,ANTOJITO...,2017-06-01,...,A,10.0
4,BANGIA,2017-06-16,...,A,9.0
...,...,...,...,...,...
95,VALL'S P...,2017-03-15,...,A,9.0
96,VIP GRILL,2017-06-12,...,A,10.0
97,WAHIZZA,2017-04-13,...,A,10.0
98,WANG MAN...,2017-08-29,...,A,12.0


### How it works...

### There's more...

In [71]:
(inspections
    .pivot_table(index=['Name', 'Date'],
                 columns='Info',
                 values='Value',
                 aggfunc='first') 
    .reset_index() 
    .rename_axis(None, axis='columns')
)

,Name,Date,...,Grade,Score
0,3 STAR J...,2017-05-10,...,A,12.0
1,A & L PI...,2017-08-22,...,A,9.0
2,AKSARAY ...,2017-07-25,...,A,13.0
3,ANTOJITO...,2017-06-01,...,A,10.0
4,BANGIA,2017-06-16,...,A,9.0
...,...,...,...,...,...
95,VALL'S P...,2017-03-15,...,A,9.0
96,VIP GRILL,2017-06-12,...,A,10.0
97,WAHIZZA,2017-04-13,...,A,10.0
98,WANG MAN...,2017-08-29,...,A,12.0


## Tidying when two or more values are stored in the same cell

### How to do it..

In [72]:
cities = pd.read_csv('data/texas_cities.csv')
cities

,City,Geolocation
0,Houston,29.7604°...
1,Dallas,32.7767°...
2,Austin,30.2672°...


In [73]:
geolocations = cities.Geolocation.str.split(pat='. ',
    expand=True)
geolocations.columns = ['latitude', 'latitude direction',
    'longitude', 'longitude direction']

In [74]:
geolocations = geolocations.astype({'latitude':'float',
   'longitude':'float'})
geolocations.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [75]:
(geolocations
    .assign(city=cities['City'])
)

,latitude,latitude direction,...,longitude direction,city
0,29.7604,N,...,W,Houston
1,32.7767,N,...,W,Dallas
2,30.2672,N,...,W,Austin


### How it works...

In [76]:
geolocations.apply(pd.to_numeric, errors='ignore')

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


### There's more...

In [77]:
cities.Geolocation.str.split(pat=r'° |, ', expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [78]:
cities.Geolocation.str.extract(r'([0-9.]+). (N|S), ([0-9.]+). (E|W)',
   expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


## Tidying when variables are stored in column names and values

### Getting ready

In [79]:
sensors = pd.read_csv('data/sensors.csv')
sensors

,Group,Property,...,2015,2016
0,A,Pressure,...,973,870
1,A,Temperature,...,1036,1042
2,A,Flow,...,882,856
3,B,Pressure,...,806,942
4,B,Temperature,...,1002,1013
5,B,Flow,...,824,873


In [80]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year')

,Group,Property,Year,value
0,A,Pressure,2012,928
1,A,Temperature,2012,1026
2,A,Flow,2012,819
3,B,Pressure,2012,817
4,B,Temperature,2012,1008
...,...,...,...,...
25,A,Temperature,2016,1042
26,A,Flow,2016,856
27,B,Pressure,2016,942
28,B,Temperature,2016,1013


In [81]:
(sensors
    .melt(id_vars=['Group', 'Property'], var_name='Year') 
    .pivot_table(index=['Group', 'Year'],
                 columns='Property', values='value') 
    .reset_index() 
    .rename_axis(None, axis='columns')
)

,Group,Year,...,Pressure,Temperature
0,A,2012,...,928,1026
1,A,2013,...,873,1038
2,A,2014,...,814,1009
3,A,2015,...,973,1036
4,A,2016,...,870,1042
5,B,2012,...,817,1008
6,B,2013,...,877,1041
7,B,2014,...,914,1009
8,B,2015,...,806,1002
9,B,2016,...,942,1013


### How it works...

### There's more...

In [82]:
(sensors
    .set_index(['Group', 'Property']) 
    .stack() 
    .unstack('Property') 
    .rename_axis(['Group', 'Year'], axis='index') 
    .rename_axis(None, axis='columns') 
    .reset_index()
)

,Group,Year,...,Pressure,Temperature
0,A,2012,...,928,1026
1,A,2013,...,873,1038
2,A,2014,...,814,1009
3,A,2015,...,973,1036
4,A,2016,...,870,1042
5,B,2012,...,817,1008
6,B,2013,...,877,1041
7,B,2014,...,914,1009
8,B,2015,...,806,1002
9,B,2016,...,942,1013
